<a href="https://colab.research.google.com/github/Ohnoimded/gaia-object-classification/blob/main/Gaia_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

url = "http://cdsxmatch.u-strasbg.fr/xmatch/uws/xmatch?sessionId=85FC152E9A0023B807E77F59A9D90A45&command=getResult&jobId=1681024617589A&resultFormat=csv"
filename = "data.csv"

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(filename, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"File downloaded successfully as {filename}")


In [ ]:
import dask.dataframe as dd
from dask.delayed import delayed
df=dd.read_csv('data.csv',blocksize=64000000, assume_missing=True,sample=100000)

In [ ]:
cols = ['Plx', 'RPlx', 'PM', 'pmDE', 'RADEcor', 'RAPlxcor', 'RApmRAcor', 'RApmDEcor', 
        'DEPlxcor', 'DEpmRAcor', 'DEpmDEcor', 'PlxpmRAcor', 'PlxpmDEcor', 'pmRApmDEcor', 
        'epsi', 'sepsi', 'nueff', 'amax', 'MatchObs', 'IPDgofha', 'IPDfow', 'RUWE', 
        'o_Gmag', 'FG', 'RFG', 'Gmag', 'o_BPmag', 'FBP', 'RFBP', 'BPmag', 'o_RPmag', 
        'FRP', 'RFRP', 'RPmag', 'E(BP/RP)', 'NBPcont', 'NBPblend', 'NRPcont', 'NRPblend', 
        'BP-RP', 'BP-G', 'G-RP']

# Define the cleaning function
@delayed
def clean_column(col):
    return col[(np.abs(stats.zscore(col)) < 5)]

# Loop through the columns and apply the cleaning function using Dask delayed
for col in cols:
    df[col] = df[col].map(clean_column, meta=('clean', 'f8'))

# Compute the cleaned DataFrame
ddf_clean = df.compute()